In [ ]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2 import service_account
import io
import json
import pandas as pd
import pandasql as psql
import zipfile
import regex
from collections import Counter


ID ="1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis"
CREDS_PATH = "../develop/creds.json"
FILENAME = "tweets"


with open(CREDS_PATH, 'r') as file:
    credz = json.load(file)


def download_file(credz, file_name):
    credentials = service_account.Credentials.from_service_account_info(credz)
    drive_service = build('drive', 'v3', credentials=credentials)

    request = drive_service.files().get_media(fileId=ID)
    #fh = io.BytesIO() # this can be used to keep in memory
    fh = io.FileIO(f'{file_name}.zip', 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()


def convert_to_parquet(file_name):
    # Replace 'file_path.zip' with the path to your zipped file
    with zipfile.ZipFile(f'{file_name}.zip', 'r') as z:
        # Assuming there's only one file in the zip archive
        file_in_zip = z.namelist()[0]
        with z.open(file_in_zip) as f:
            # Load the JSON file into a DataFrame
            df = pd.read_json(f, lines=True)
    df.to_parquet(f'{file_name}.parquet')


download_file(credz, FILENAME)
convert_to_parquet(FILENAME)
df = pd.read_parquet(f'../develop/{FILENAME}.parquet')